# Лабораторная работа №2
## Выполнил студент группы БФИ2001 Стеклов М. А.

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)

### Задание №1
Реализовать методы поиска в соответствии с заданием. Организовать генерацию начального набора случайных данных. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [21]:
from random import randint
def get_rand_list(n: int, min_value:int, max_value:int) -> list[int]:
    return [randint(min_value, max_value) for _ in range(n)]

In [22]:
x = get_rand_list(10, -100, 100)
sorted_x = sorted(x)

In [35]:
#Бинарный поиск
def binary_search_insertion_index(x: list[int], el: int) -> int:
    n = len(x)
    l, r = 0, n - 1
    m = (l + r) // 2
    while r - l > 1:
        if el <= x[m]:
            r = m
        else:
            l = m
        m = (l + r) // 2
    return r

# Поиск
def binary_search_find(x: list[int], el: int) -> int:
    i = binary_search_insertion_index(x, el)
    assert el == x[i], "Element not found"
    return i
    

# Вставка
def binary_search_insert(x: list[int], el: int) -> list[int]:
    x.insert(binary_search_insertion_index(x, el), el)
    return x


# Удаление
def binary_search_remove(x: list[int], el: int) -> list[int]:
    i = binary_search_find(x, el)
    x.pop(i)
    return x

AssertionError: Element not found

0 16
0 8
0 4
2 4
[-72, -41, 8, 10, 13, 15, 15, 15, 15, 15, 16, 34, 35, 38, 44, 53, 79, 87]


In [38]:
#Бинарное дерево

class BinarySearchTree:
    def __init__(self, key: int=None, value: int=None):
        self.left: BinaryTree = None
        self.right: BinaryTree = None
        if key is None or value is None:
            self.key = None
            self.value = None
        else:
            self.key = key
            self.value = value
            
    def find(self, key: int) -> BinarySearchTree:
        assert key is not None, "Key can't be None"
        assert self.key is not None, f"Element with key {key} not found"
        if key == self.key:
            return self
        return self.left.find(key) if key < self.key else self.right.find(key)
    
    def insert(self, key: int, value: int):
        assert key is not None, "Key can't be None"
        if self.key is None or self.key == key:
            self.key = key 
            self.value = value
            return
        if self.left is None:
            self.left = BinarySearchTree()
        if self.right is None:
            self.right = BinarySearchTree()
        if key < self.key:
            self.left.insert(key, value)
        else:
            self.right.insert(key, value)
    
    def remove(self, key: int):
        assert key is not None, "Key can't be None"
        assert self.key is not None, f"Element with key {key} not found"
        if key < self.key:
            self.left.remove(key)
        elif key > self. key:
            self.right.remove(key)
        else:
            if self.left.key is None:
                if self.right.key is None:
                    self.key = None
                    self.value = None
                    self.left = None
                    self.right = None
                else:
                    self = self.right
            else
                if self.right.key is None:
                    self = self.left 
                else:
                    self = 
    def __str__(self):
        pass
        

SyntaxError: only single target (not tuple) can be annotated (412711623.py, line 5)

In [ ]:
#Метод Фибоначчи

In [ ]:
#Интерполяционный

### Задание №2
Написать соответствующие решения для заданных методов.

In [ ]:
#Простое рехэширование

In [ ]:
#Рехэширование с помощью псевдослучайных чисел


In [ ]:
#Метод цепочек

### Задание №3
Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям
Написать программу,  которая находит хотя бы один способ решения задач.


In [5]:
Board = set[tuple[int, int]]
def helper(board: Board, depth=0) -> tuple[bool, list[tuple[int, int]]]:
    if depth == 8:
        return (True, [])
    if len(board) == 0:
        return (False, [])
    my_board = board.copy()
    for i, j in board:
        my_board -= {(i, _j) for _j in range(8)}
        my_board -= {(_i, j) for _i in range(8)}
        d = i - j
        i_s, j_s = (0, -d) if d <= 0 else (d, 0)
        my_board -= {(i_s + s, j_s + s) for s in range(min(8 - i_s, 8 - j_s))}
        d = 7 - j - i
        i_s, j_s = (-d, 7) if d <= 0 else (0, 7 - d)
        my_board -= {(i_s + s, j_s - s) for s in range(min(8 - i_s, j_s))}
        if (res := helper(my_board, depth=depth + 1))[0]:
            res[1].append((i, j))
            return res
        my_board = board.copy()
    return (False, [])

from IPython.display import display, Markdown
def task3():
    board = {(i, j) for i in range(8) for j in range(8)}
    board = helper(board)[1]
    out_board = [[(lambda y, x: '▢' if (i + j) % 2 == 0 else '▣')(i, j)
                  for j in range(8)] for i in range(8)]
    for i, j in board:
        out_board[i][j] = '♕' if (i + j) % 2 == 0 else '♛'
    display(Markdown("<br>".join(map(lambda x: f"""<font size="5">{' '.join(x)}</font>""", out_board))))
task3()

<font size="5">▢ ▣ ♕ ▣ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ♛ ▢ ▣ ▢</font><br><font size="5">▢ ♛ ▢ ▣ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ▣ ▢ ▣ ♕</font><br><font size="5">♕ ▣ ▢ ▣ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ▣ ▢ ♛ ▢</font><br><font size="5">▢ ▣ ▢ ♛ ▢ ▣ ▢ ▣</font><br><font size="5">▣ ▢ ▣ ▢ ▣ ♕ ▣ ▢</font>